<a href="https://colab.research.google.com/github/jcmachicao/gdmk_uc__reportes_edusights/blob/main/ucont__mprecitivo_dataprep_202502.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento de Red Neuronal Clasificación Binaria PyTorch

## Preparación

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from torchvision import datasets, transforms
import os

In [ ]:
data_path = 'drive/My Drive/00 Aplicativos/data_aplicativos/archivos/'
os.listdir(data_path)

['df_Ingresantes_uvir_2220al2410_0107.pkl',
 'df_Ingresantes_uvir_2220al2410_abandono_mas_deserción.pkl',
 'df_Ingresantes_uvir_2220al2410_deserción.pkl',
 'Estudiantes_202420_al1107.xlsx',
 'X_train.pt',
 'y_train.pt',
 'X_test.pt',
 'y_test.pt']

### Carga de datos originales

In [ ]:
#data_file = 'df_Ingresantes_uvir_2220al2410_0107.pkl'
data_file = 'df_Ingresantes_uvir_2220al2410_deserción.pkl'
df = pd.read_pickle(data_path+data_file).dropna()
df.shape, df.columns

((8020, 27),
 Index(['Tiempo_espera', 'DNI', 'NOMBRE', 'TIPO_INGRESANTE',
        'ESTADO_INGRESANTE', 'APROBO_RV', 'APROBO_RM', 'Puntaje_Admisión',
        'Edad', 'Rango_Edad', 'Sexo', 'Dep_viv_act', 'Dis_viv_act', 'Migrante',
        'Estado Civil', 'Dependientes', 'Vives_con', 'Tipo_de_cole',
        'Max_Grado', 'Estudio_Madre', 'Estudio_Padre', 'Tipo_lugar_origen',
        'Ocupación', 'Tipo_trabajo', 'Ingre_estudiante', 'Ingre_familia',
        'Dispo_Internet'],
       dtype='object'))

In [ ]:
df.ESTADO_INGRESANTE.value_counts()

,count
ESTADO_INGRESANTE,
Activo,4302
Abandono,2763
Abandono_real,955


In [ ]:
df.shape

(8020, 27)

In [ ]:
MAX_CAT = 40

In [ ]:
def selec_categ(df, MAX_CAT):

  df.index = df.DNI
  df.drop(columns=['DNI'], inplace=True)
  print(df.shape)
  cat_list = []
  num_list = []
  for col in df.columns:
    if df[col].dtype == 'object':
      if len(df[col].unique()) < MAX_CAT:
        #print(col, list(df[col].unique()))
        cat_list.append(col)
      else:
        pass
    else:
      num_list.append(col)
  df_maxcat = df[cat_list]
  df_num = df[num_list]
  return df_maxcat, df_num

In [ ]:
df_cat, df_num = selec_categ(df, MAX_CAT)
df_cat.shape, df_num.shape

(8020, 26)


((8020, 22), (8020, 2))

In [ ]:
df_cat.shape, df_num.shape

((8020, 22), (8020, 2))

In [ ]:
def genera_tensor(df_cat, df_num):

  df_b = pd.get_dummies(df_cat)
  df_b = df_b.set_index(df_cat.index)
  print(df_b.index[0:5])
  scaler = MinMaxScaler()
  X_sc = scaler.fit_transform(df_b)
  df_sc = pd.DataFrame(X_sc)
  df_sc.columns = df_b.columns
  df_n = df_num
  df_n = df_n.set_index(df_b.index)
  print(df_n.index[0:5])
  df_sc.index = df_b.index
  print((df_sc.index == df_n.index).sum()==len(df_b))
  df_r = pd.concat([df_sc, df_n], axis=1)
  df_r.index = df_b.index
  return df_r

In [ ]:
df_rr = genera_tensor(df_cat, df_num)
df_rr.shape

Index(['60971965', '81635783', '81704629', 'tm898474', '76508439'], dtype='object', name='DNI')
Index(['60971965', '81635783', '81704629', 'tm898474', '76508439'], dtype='object', name='DNI')
True


(8020, 152)

In [ ]:
df_rr.drop(columns=['ESTADO_INGRESANTE_Abandono', 'ESTADO_INGRESANTE_Abandono_real'], inplace=True)
df_rr.columns[0:30]

Index(['Tiempo_espera_Entre 31 a 60 días', 'Tiempo_espera_Entre 61 a 90 días',
       'Tiempo_espera_Menos de 30 días', 'Tiempo_espera_Más de 91 días',
       'TIPO_INGRESANTE_CONVALIDANTE', 'TIPO_INGRESANTE_MOVILIDAD',
       'TIPO_INGRESANTE_ORDINARIO', 'ESTADO_INGRESANTE_Activo', 'APROBO_RV_NO',
       'APROBO_RV_SI', 'APROBO_RM_NO', 'APROBO_RM_SI',
       'Rango_Edad_De  41 a 50', 'Rango_Edad_De 15 a 21',
       'Rango_Edad_De 22 a 25', 'Rango_Edad_De 26 a 31',
       'Rango_Edad_De 32 a 40', 'Rango_Edad_De 51 a 60',
       'Rango_Edad_De 61 a 80', 'Sexo_Femenino', 'Sexo_Masculino',
       'Dep_viv_act_AMAZONAS', 'Dep_viv_act_ANCASH', 'Dep_viv_act_APURIMAC',
       'Dep_viv_act_AREQUIPA', 'Dep_viv_act_AYACUCHO', 'Dep_viv_act_Alemania',
       'Dep_viv_act_Argentina', 'Dep_viv_act_Brasil', 'Dep_viv_act_CAJAMARCA'],
      dtype='object')

In [ ]:
df_rr.shape

(8020, 150)

In [ ]:
df_rr.to_pickle(data_path + 'df_rr_S2.pkl')

In [ ]:
lista_categorias = list(df_rr.columns)
lista_categorias.remove('ESTADO_INGRESANTE_Activo')
list_df = pd.DataFrame(lista_categorias)
list_df.to_excel(data_path + 'lista_categorias_S2_CON_69.xlsx')
list_df.shape

(149, 1)

In [ ]:
df_rr['ESTADO_INGRESANTE_Activo'].value_counts()

,count
ESTADO_INGRESANTE_Activo,
1.0,4302
0.0,3718
